<a href="https://colab.research.google.com/github/tenexcoder/huggingface-tutorials/blob/main/BERT_tokenizer_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you're opening this Notebook on colab, you will probably need to install 🤗 Tokenizers. Uncomment the following cell and run it.


In [ ]:
#! pip install tokenizers===0.9.3

     |████████████████████████████████| 2.9MB 2.7MB/s 


If you're opening this notebook locally, make sure your environment has an install from source for both those libraries.

## Prepare the dataset

In [ ]:
# first off we create the data/ dir, download raw wiki-103, and finally unzip the file
!mkdir data
!wget https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip -P data
!unzip data/wikitext-103-raw-v1.zip -d data

mkdir: cannot create directory ‘data’: File exists
--2020-11-07 00:20:54--  https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.136.29
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.136.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191984949 (183M) [application/zip]
Saving to: ‘data/wikitext-103-raw-v1.zip.1’

wikitext-103-raw-v1 100%[===================>] 183.09M  75.7MB/s    in 2.4s    

2020-11-07 00:20:57 (75.7 MB/s) - ‘data/wikitext-103-raw-v1.zip.1’ saved [191984949/191984949]

Archive:  data/wikitext-103-raw-v1.zip
   creating: data/wikitext-103-raw/
  inflating: data/wikitext-103-raw/wiki.test.raw  
  inflating: data/wikitext-103-raw/wiki.valid.raw  
  inflating: data/wikitext-103-raw/wiki.train.raw  


## Tokenizer from scratch

First, BERT relies on WordPiece, so we instantiate a new Tokenizer with this model:

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece

bert_tokenizer = Tokenizer(WordPiece())

Then we know that BERT preprocesses texts by removing accents and lowercasing. We also use a unicode normalizer:

In [ ]:
from tokenizers import normalizers
from tokenizers.normalizers import Lowercase, NFD, StripAccents

bert_tokenizer.normalizer = normalizers.Sequence([NFD(), Lowercase(), StripAccents()])

The pre-tokenizer is just splitting on whitespace and punctuation:

In [ ]:
from tokenizers.pre_tokenizers import Whitespace

bert_tokenizer.pre_tokenizer = Whitespace()

And the post-processing uses the template we saw in the previous section:

In [ ]:
from tokenizers.processors import TemplateProcessing

bert_tokenizer.post_processor = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", 1),
        ("[SEP]", 2),
    ],
)

We can use this tokenizer and train on it on wikitext like in the [Quicktour](https://huggingface.co/docs/tokenizers/python/latest/quicktour.html):

In [ ]:
from tokenizers.trainers import WordPieceTrainer

trainer = WordPieceTrainer(
    vocab_size=30522, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)
files = [f"data/wikitext-103-raw/wiki.{split}.raw" for split in ["test", "train", "valid"]]
bert_tokenizer.train(trainer, files)

model_files = bert_tokenizer.model.save("data", "bert-wiki")
bert_tokenizer.model = WordPiece.from_file(*model_files, unk_token="[UNK]")

bert_tokenizer.save("data/bert-wiki.json")

### Decoding

On top of encoding the input texts, a `Tokenizer` also has an API for decoding, that is converting IDs generated by your model back to a text. This is done by the methods `decode()` (for one predicted text) and `decode_batch()` (for a batch of predictions).

The decoder will first convert the IDs back to tokens (using the tokenizer’s vocabulary) and remove all special tokens, then join those tokens with spaces:

In [ ]:
output = bert_tokenizer.encode("Hello, y'all! How are you 😁 ?")
print(output.ids)
# [1, 27462, 16, 67, 11, 7323, 5, 7510, 7268, 7989, 0, 35, 2]

bert_tokenizer.decode([1, 27462, 16, 67, 11, 7323, 5, 7510, 7268, 7989, 0, 35, 2])
# "Hello , y ' all ! How are you ?"

[1, 27462, 16, 67, 11, 7323, 5, 7510, 7268, 7989, 0, 35, 2]


"hello , y ' all ! how are you ?"

If you used a model that added special characters to represent subtokens of a given “word” (like the `"##"` in WordPiece) you will need to customize the decoder to treat them properly. If we take our previous `bert_tokenizer` for instance the default decoing will give:

In [ ]:
output = bert_tokenizer.encode("Welcome to the 🤗 Tokenizers library.")
print(output.tokens)
# ["[CLS]", "welcome", "to", "the", "[UNK]", "tok", "##eni", "##zer", "##s", "library", ".", "[SEP]"]

bert_tokenizer.decode(output.ids)
# "welcome to the tok ##eni ##zer ##s library ."

['[CLS]', 'welcome', 'to', 'the', '[UNK]', 'tok', '##eni', '##zer', '##s', 'library', '.', '[SEP]']


'welcome to the tok ##eni ##zer ##s library .'

But by changing it to a proper decoder, we get:

In [ ]:
from tokenizers import decoders

bert_tokenizer.decoder = decoders.WordPiece()
bert_tokenizer.decode(output.ids)
# "welcome to the tokenizers library."

'welcome to the tokenizers library.'